In [507]:
#modules
#!pip install requests -q
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from time import time

In [576]:
#Create dictionaries of search parameters
#Later on, add values to this as you get more job titles from search results and we can phase this out boo :)
titles = [
    'Data Scientist', 'Data', 'BI', 'Decision Science', 'ML', 'Machine Learning', 'Applied Scientist', 
    'Modeling Analyst', 'Business Analyst', 'Business Intelligence', 'Analyst',  'Developer', 'Scientist', 
    'Quantitative', 'Artificial Intelligence', 'AI', 'Engineer', 'Big-Data', 'Software Engineer'
]
size = len(titles)
#search result job titles
addon_titles = []

qualifiers = ['Intern', 'Junior']
for t in titles[:size]: 
    titles.extend([qualifiers[0]+'-'+ t, qualifiers[1]+'-' + t])
    
locationDict = {
    0: 'Pittsburgh, PA',  1: 'Nashville, TN',  2: 'California',  3: 'Austin, TX',  4: 'Minneapolis, MN',  
    5: 'San Antonio, TX'
}

In [577]:
#description corpus i.e. each description is a document
#alltext = desc
sk = ['python', 'pyspark', 'numpy', 'pandas', 'nltk', 'gensim', 'scikit-learn', 'sklearn', 'nlp', 'tensorflow',
     'machine learning', 'AI', 'BI', 'ML', 'analysis', 'excel', 'vba', 'C#', 'Java', 'STATA', 'EVIEWS',
     'deep learning', 'master\'s', 'masters', 'bachelor\'s', 'bachelors', 'models']

immigration = ['immigration', 'W2', 'sponsorship', 'visa', 'citizenship status']

In [578]:
#class for each job posting
class Job_Posting:
    #attributes
    
    def __init__(self): pass
    
    def __init__(self, title, company, location, date, skills, visa, relo):
        self.job_title = title
        self.company_name = company
        self.job_location = location
        self.posted_date = date
        self.tech_skills = self.get_skills_requested(skills)
        self.immigration_assistance = visa
        self.relocation_assistance = relo
        
    def get_skills_requested(self, description):
        skills = []
        #model, work in progress. Will move here once finalized for phase 1
        return description

In [579]:
#method to get search results for dictionary positions and all locations before filtering
#Posted in the last 24 hours and <= 10 miles from job location
def get_all_location_results(job_titles, location): 
    end = len(job_titles)
    links = []
    print(end, 'search terms: \n-----------------------------\n')
    for title in job_titles:
        URL = 'https://www.linkedin.com/jobs/search?keywords='+ title +'&location='+location+'&f_TP=1'
        page = requests.get(URL)
        soup = bs(page.content, 'lxml')
        refs = soup.find_all('a', class_='result-card__full-card-link')
        links += [ref.get('href') for ref in refs if ref.get('href') not in links]
    print(location + ':', len(links), 'result links')
    return links

In [608]:
titles = [
    'Data Scientist', 'Data', 'BI', 'Decision Science', 'ML', 'Machine Learning', 'Applied Scientist', 
    'Modeling Analyst', 'Business Analyst', 'Business Intelligence', 'Analyst',  'Developer', 'Scientist', 
    'Quantitative', 'Artificial Intelligence', 'AI', 'Engineer', 'Big-Data', 'Software Engineer'
]
#method to get the info in the links
def get_job_postings(links, titles):
    end = len(links)
    print('Number of links to start:', end)
    postings = []
    job_titles = []#new stuff
    for link in links:
        #get page data
        page = requests.get(link)
        soup = bs(page.content, 'lxml')
        job = soup.find('h1', 'topcard__title')
        if job is not None:
            job = re.sub(r'[^\w\s]','', job.get_text().strip())
            if job.title() not in titles: job_titles.append(job.title())
            company = soup.find('a', 'topcard__org-name-link topcard__flavor--black-link')
            if company is not None: company = company.get_text().strip()
            location = soup.find('span', 'topcard__flavor topcard__flavor--bullet')
            if location is not None: location = location.get_text().strip()
            posted = datetime.now().strftime('%Y-%m-%d')
            skillsDesc = soup.find('div', 'description__text description__text--rich')
            if skillsDesc is not None: skillsDesc = skillsDesc.get_text()
            #check to see for desired locations
            loc_list = ' '.join(locationDict.values())
            if loc_list.find(location) != -1:
                posting = Job_Posting(job, company, location, posted, skillsDesc, False, False)
                postings.append(posting)
    print('Today\'s new job titles:', len(job_titles))     
    titles += job_titles
    return postings

In [609]:
#method to tokenize and pre-process job descriptions
def tokenize_job_descriptions(postings):
    word_freq = defaultdict(int)
    descs = []
    for posting in postings:
        desc = posting.tech_skills
        if desc is not None:
            desc = desc.lower()
            desc = re.sub(r'[^c#\w\s]', ' ', desc)
            desc = desc.split()
            desc = [word for word in desc if word not in stopwords.words('english')]
            desc = [word for word in desc if len(word) > 1 or (len(word) == 1 and word =='r')]
            desc.append(posting.job_title)
            for word in desc:
                word_freq[word] += 1
            descs.append(desc)
    print('Total unique tokens from list:', len(word_freq))
    return descs

In [610]:
t = time()
links = []
links += get_all_location_results(titles, 'United-States')
print('Time to get search results: {m}:{s:02d} mins'.format(m = round((time() - t )/ 60), s = round((time()-t)%60)))

19 search terms: 
-----------------------------

United-States: 473 result links
Time to get search results: 0:15 mins


In [611]:
t = time()
postings = get_job_postings(links, titles)
print('Total job postings:', len(postings))
print('Time to extract link info: {m}:{s:02d} mins'.format(m = round((time() - t )/ 60), s = round((time()-t)%60)))

Number of links to start: 473
Today's new job titles: 389
Total job postings: 16
Time to extract link info: 6:41 mins


In [612]:
t = time()
descs = tokenize_job_descriptions(postings)
print('Time to tokenize values: {m}:{s:02d} mins'.format(m = round((time() - t )/ 60), s = round((time()-t)%60)))

Total unique tokens from list: 978
Time to tokenize values: 0:06 mins


In [613]:
print('Links:', len(links), 'postings:', len(postings))
print('descriptions:',len(descs))

Links: 473 postings: 16
descriptions: 16


 ### Gensim Models

In [614]:
#modules
from collections import defaultdict
from gensim.models import Word2Vec

In [623]:
#gensim model
#size: The number of dimensions of the embeddings and the default is 100.
#window: The maximum distance between a target word and words around the target word. The default window is 5.
#min_count: The minimum count of words to consider when training the model; words with occurrence less than this count will be ignored. The default for min_count is 5.
#workers: The number of partitions during training and the default workers is 3.
#sg: The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW.
model = Word2Vec(min_count=1, size= 50,workers=3, window =3, sg = 1, alpha=0.03, min_alpha=0.0007)

In [624]:
from time import time
t = time()
model.build_vocab(descs, progress_per=10000)
print('Time to build vocab: {m}:{s:02d} mins'.format(m = round((time() - t )/ 60), s = round((time()-t)%60)))

Time to build vocab: 0:01 mins


In [625]:
t = time()
size = int(len(descs)*0.8)
model.train(descs[:size], total_examples=model.corpus_count, epochs=30, report_delay=1)
print('Time to train model: {m}:{s:02d} mins'.format(m = round((time() - t )/ 60), s = round((time()-t)%60)))

Time to train model: 0:01 mins


In [626]:
#make model more memory efficient
model.init_sims(replace=True)

In [627]:
#what words are most similar to this
model.wv.most_similar(positive=['python'])

[('java', 0.9938955307006836),
 ('scala', 0.991338849067688),
 ('one', 0.9663410186767578),
 ('equivalent', 0.9582749605178833),
 ('qualificationsmastery', 0.9481650590896606),
 ('sparkexperience', 0.9437363147735596),
 ('languageexperience', 0.9403271675109863),
 ('language', 0.9319831132888794),
 ('scalaexperience', 0.9282258749008179),
 ('libraries', 0.9249277114868164)]

In [628]:
#how similar are these two
model.wv.similarity('python', 'spark')

0.8321775

In [629]:
#odd one out
model.wv.doesnt_match(['master', 'python', 'r'])

'r'

In [630]:
#master is to degree as python is to....?
model.wv.most_similar(positive=['master', 'python'], negative=['degree'], topn=5)

[('scala', 0.9402890205383301),
 ('qualificationsmastery', 0.9359278678894043),
 ('equivalent', 0.9343596696853638),
 ('java', 0.9301947355270386),
 ('one', 0.9272948503494263)]

### TF IDF with sk learn trial

#modules needed
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

descs = []
#Pre processing: remove punctuation and symbols, change all to lower case, tokenize
for d in descriptions:
    d = d.lower()
    d = re.sub(r'[^\w\s]', ' ', d)
    descs.append(d)

#Create vectorizer for the descriptions
vect = TfidfVectorizer(max_df = 0.7, min_df = 3, stop_words = stopwords.words('english'))
tfidf = vect.fit_transform(descs)
#for idx in range(tfIDF[0].shape[0]):
  #  print(vect.get_feature_names()[idx], tfIDF[0][idx])

#(tfidf[0].toarray())
words = list(vect.vocabulary_.keys()) #the feature names
#use key values and keys to change the dataframe so that we can see the names of the words
keys = list(vect.vocabulary_.values()) #the number representing the wordID in the tfidf vectors (docID, wordID)
values = vect.vocabulary_

for word, key in values.items(): #every word
    scores = scores.rename(columns = {key : word})

#row index = word keyID
#column index = document ID
scores = pd.DataFrame(tfidf.toarray())
x = scores[3][0] #column index, row index (in that order)

a = scores.sum(axis=0) #all values tfidf sums
b = scores.sum(axis=0).index #all words/index
c = scores.sum(axis=0).values #all sums

Tomorrow Inshallah: create small methods to parse through a string to look for substring before saving job posting:
- Only save posting if relo assistance is offered
- Only save posting if immigration is offered
- Send list of job posts to excel and download/add to it
- add link of listing to the class

In [554]:
s = 'data science'
ss = 'data'
if s.find(ss) != -1:
    print('yes')
else: print('no')

yes


In [566]:
loc_list = 
loc_list

'Pittsburgh Nashville California Austin Minneapolis San Antonio'